In [ ]:
# Setup
import os
from fastcore.all import *
from fastai.vision.all import *

In [ ]:
# Initialize dataset
path = ('/Users/joo/Developer/jupyternotebooks/LesPaulCustomIdentifier/datasets/guitars/lespaul')

In [ ]:
# Verify dataset
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
# Create DataBlock out of data
guitars = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
# Augment data for training
guitars = guitars.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = guitars.dataloaders(path)
dls.show_batch()

In [ ]:
# Train model
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(10)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
# Export trained model
learn.export('model.pkl')

In [ ]:
# Test model
custom = PILImage.create('/Users/joo/Developer/jupyternotebooks/LesPaulCustomIdentifier/datasets/guitars/tests/example.jpeg')
custom.show()
is_custom,_,probs = learn.predict(custom)
print(f"This is a Les Paul {str(is_custom).capitalize()}.")
print(f"Probability it's a Les Paul Custom: {probs[0]:.4f}")